In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string


In [17]:
from google.colab import drive
drive.mount('\content\drive')

TIMEOUT: ignored

In [15]:
data_fake = pd.read_csv('/content/drive/MyDrive/Fake.csv')
data_true = pd.read_csv('/content/drive/MyDrive/True.csv')

In [18]:
data_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [19]:
data_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [20]:
data_fake["class"]=0
data_true["class"]=1

In [21]:
data_fake.shape,data_true.shape

((23481, 5), (21417, 5))

In [22]:
data_fake_manual_testing = data_fake.tail(10)
for i in range(23480,23470,-1):
    data_fake.drop([i],axis=0,inplace=True)

data_true_manual_testing = data_true.tail(10)
for i in range(21416,21406,-1):
    data_true.drop([i],axis=0,inplace=True)

In [23]:
data_fake.shape,data_true.shape

((23471, 5), (21407, 5))

In [24]:
data_fake_manual_testing['class']=0
data_true_manual_testing['class']=1

<ipython-input-24-350d73592787>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fake_manual_testing['class']=0
<ipython-input-24-350d73592787>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_true_manual_testing['class']=1


In [25]:
data_fake_manual_testing.head()

,title,text,subject,date,class
23471,Seven Iranians freed in the prisoner swap have...,"21st Century Wire says This week, the historic...",Middle-east,"January 20, 2016",0
23472,#Hashtag Hell & The Fake Left,By Dady Chery and Gilbert MercierAll writers ...,Middle-east,"January 19, 2016",0
23473,Astroturfing: Journalist Reveals Brainwashing ...,Vic Bishop Waking TimesOur reality is carefull...,Middle-east,"January 19, 2016",0
23474,The New American Century: An Era of Fraud,Paul Craig RobertsIn the last years of the 20t...,Middle-east,"January 19, 2016",0
23475,Hillary Clinton: ‘Israel First’ (and no peace ...,Robert Fantina CounterpunchAlthough the United...,Middle-east,"January 18, 2016",0


In [26]:
data_true_manual_testing.head()

,title,text,subject,date,class
21407,"Mata Pires, owner of embattled Brazil builder ...","SAO PAULO (Reuters) - Cesar Mata Pires, the ow...",worldnews,"August 22, 2017",1
21408,"U.S., North Korea clash at U.N. forum over nuc...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1
21409,"U.S., North Korea clash at U.N. arms forum on ...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1
21410,Headless torso could belong to submarine journ...,COPENHAGEN (Reuters) - Danish police said on T...,worldnews,"August 22, 2017",1
21411,North Korea shipments to Syria chemical arms a...,UNITED NATIONS (Reuters) - Two North Korean sh...,worldnews,"August 21, 2017",1


In [27]:
data_merge = pd.concat([data_fake,data_true],axis=0)

In [28]:
data_merge.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [29]:
data_merge.shape

(44878, 5)

In [30]:
data_merge.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

In [31]:
data = data_merge.drop(['title','subject','date'],axis=1)

In [32]:
data.isnull().sum()

text     0
class    0
dtype: int64

In [33]:
data = data.sample(frac=1)

In [34]:
data.head()

,text,class
7726,"For the longest time, Donald Trump has been ad...",0
7962,CHICAGO (Reuters) - U.S. Democratic presidenti...,1
16805,Skipping customs at JFK might seem like a grea...,0
20946,"DHAKA (Reuters) - Bangladesh, one of the world...",1
1066,The Senate Republicans have attracted loads of...,0


In [35]:
data.reset_index(inplace=True)
data.drop(['index'],axis=1,inplace=True)

In [36]:
data.columns

Index(['text', 'class'], dtype='object')

In [37]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]','',text)
    text = re.sub('\\W','',text)
    text = re.sub('https?://\S+','',text)
    text = re.sub('<.*?>+','',text)
    text = re.sub('[%s]' % re.escape(string.punctuation),'',text)
    text = re.sub('\n','',text)
    text = re.sub('w*\d\w*','',text)
    return text

In [38]:
data['text'] = data['text'].apply(wordopt)

In [39]:
x = data['text']
y = data['class']

In [41]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25)

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [43]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

lr.fit(xv_train,y_train)



LogisticRegression()

In [44]:
pred_lr = lr.predict(xv_test)

In [45]:
lr.score(xv_test,y_test)

0.551336898395722

In [46]:
print(classification_report(y_test,pred_lr))

              precision    recall  f1-score   support

           0       0.54      1.00      0.70      5939
           1       1.00      0.05      0.09      5281

    accuracy                           0.55     11220
   macro avg       0.77      0.52      0.40     11220
weighted avg       0.76      0.55      0.41     11220



In [47]:
from sklearn.tree import DecisionTreeClassifier
DT = DecisionTreeClassifier()
DT.fit(xv_train,y_train)

DecisionTreeClassifier()

In [48]:
predict_dt = DT.predict(xv_test)

In [49]:
DT.score(xv_test,y_test)

0.551336898395722

In [57]:
print(classification_report(y_test,predict_dt))

              precision    recall  f1-score   support

           0       0.54      1.00      0.70      5939
           1       1.00      0.05      0.09      5281

    accuracy                           0.55     11220
   macro avg       0.77      0.52      0.40     11220
weighted avg       0.76      0.55      0.41     11220



In [52]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=0)
gb.fit(xv_train,y_train)

GradientBoostingClassifier(random_state=0)

In [53]:
predict_gb = gb.predict(xv_test)

In [54]:
gb.score(xv_test,y_test)

0.5307486631016043

In [58]:
print(classification_report(y_test,predict_gb))

              precision    recall  f1-score   support

           0       0.53      1.00      0.69      5939
           1       1.00      0.00      0.01      5281

    accuracy                           0.53     11220
   macro avg       0.77      0.50      0.35     11220
weighted avg       0.75      0.53      0.37     11220



In [60]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(xv_train,y_train)

RandomForestClassifier()

In [61]:
predict_rf = rf.predict(xv_test)

In [62]:
rf.score(xv_test,y_test)

0.551336898395722

In [63]:
print(classification_report(y_test,predict_rf))

              precision    recall  f1-score   support

           0       0.54      1.00      0.70      5939
           1       1.00      0.05      0.09      5281

    accuracy                           0.55     11220
   macro avg       0.77      0.52      0.40     11220
weighted avg       0.76      0.55      0.41     11220



In [73]:
def output_label(n):
  if n==0:
    return "Fake news"
  elif n==1:
    return "Not Fake"

def manual_testing(news):
    testing_news = {"text": [news]}
    new_def_test = pd.DataFrame(testing_news)


    new_def_test["text"] = new_def_test["text"].apply(wordopt)

    new_x_test = new_def_test["text"]


    new_xv_test = vectorization.transform(new_x_test)
    pred_lr = lr.predict(new_xv_test)
    predict_dt = DT.predict(new_xv_test)
    predict_gb = gb.predict(new_xv_test)
    predict_rf = rf.predict(new_xv_test)

    print("\n\nLR prediction: {} \nDT prediction: {} \nGB prediction: {} \nRF prediction: {}".format(
        output_label(pred_lr[0]),
        output_label(predict_dt[0]),
        output_label(predict_gb[0]),
        output_label(predict_rf[0])
    ))

In [ ]:
news = str(input())
manual_testing(news)
